# Estudio sobre la tienda on-line de:

## [AMANTIS](https://www.amantis.net/)

## ¿Por qué este estudio?

Primero, este es un trabajo de análisis de datos y de predicción sobre una tienda on-line. Creo que las personas que veamos esta documentación somos adultas y lo suficientemente maduras para asumir que el sexo es parte de nuestra vida.

Por este motivo, el hacer un estudio sobre una tienda on-line erótica es lo mismo que realizarlo sobre una tienda de ropa, muebles, un supermercado,...

Segundo, yo soy usuario registrado de esta tienda on-line. No solo realizo compras para mí o para mi pareja sino también para otras personas. 
Es por este motivo el que recibo periódicamente correos de ofertas de esta tienda on-line y es aquí en donde entra mi motivación personal.

Los correos que envían a sus usuarios están basados en ofertas que lanzan (estas ofertas están basadas en rebajas) y son muy arbitrarias. Suelo recibir correos de ofertas de determinados productos de los que NO he realizado una compra del mismo o similares. 

Por este motivo, considero interesante poder realizar algún tipo de estudio sobre los productos que tengan en la tienda y poder dar una mejor experiencia al comprador.

## Consideraciones para el proyecto.

Este proyecto está estructurado en 4 partes:

- Obtención y tratamiento de datos de la página web, a partir de *Webscrapping* y su manipulación para poder tratarlos a través de las librerías de *pandas* y *numpy* y su almacenamiento en una *Base de datos Relacional*, dada la posibilidad de limitar los campos a realizar y la atomización de los datos.
   - A su vez se realizará algún tipo de visionado de los datos obtenidos para ver diversos aspectos a tener en cuenta.
- Generación de nuevas variables, *featuring engineering*, para poder dar una mejor visión de los productos que ofrecen en la tienda.
    - A través de la generación de estas nuevas variables, pretendemos dar un visionario más completo sobre la relación de las diversas variables entre sí y los usuarios.
    - Estas variables pueden ser obtenidas a través de *pandas*, *RegEx* y *NLP*.
- Análisis de la información obtenida y generada para ver la relación existente entre usuarios, productos, fechas...
   - Esta información será aplicada a partir de diversas queries de la base de datos a utilizar y con las librerías de *pandas*, *matplotlib* y *seaborn*.
- Generación de modelados de los datos obtenidos para establecer gustos de los usuarios y dar sugerencia de otros productos que tengan etiquetas similares.

    


## 1. Obtención y tratamiento de los datos.

A partir de la **URL** *'https://www.amantis.net/productos-amantis/'* vamos a realizar un recorrido por las diferentes páginas que dispone y en donde están alojadas las **URLs** de los diferentes productos que disponen.

Una vez que accedemos a la dirección de cada producto, vemos que hay una serie de datos que queremos recoger de la misma:
> Nombre.
> Precio del producto.
>
> Precio rebajado del producto.
>
> Descripción e información sobre el producto (tamaño, duración de baterias, limpieza ...). 
>
> Comentarios.

Dentro de comentarios disponemos de más información, que consideramos interesante:
> Usuario.
>
> Rating.
>
> Fecha comentario.
>
> Cuerpo del comentario.


A través de la librería para Webscrapping *Beautiful Soap* procederemos a extraer esta información y en determinados casos a tratarla para poder trabajar con ella.

Para ello cargaremos las librerías necesarias.

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
url = "https://www.amantis.net/productos-amantis/"              # lista productos
url_principal="https://www.amantis.net/"                        # productos
# pages= np.arange(1,5)
pages= np.arange(1, 25)

'''Listas a generar con la información de los productos'''
lista_URLs = []
name=[]
regular_prices=[]
new_price=[]
info=[]
user_comments=[]
comment=[]
date=[]
ratings=[]
id=[]
comentarios=[]



'''Generamos 2 diccionarios con los datos importantes para ingresar en una BBDD'''

diccionario_datos_productos={"ID":id,"NAME":name,"INFO":info,"LISTA_URL":lista_URLs,"REGULAR_PRICE":regular_prices,"DISCOUNT_PRICE":new_price}

diccionario_comentarios_productos={"ID":id,"COMENTARIOS":comentarios}



''' Obtenemos las URLs de los productos para entrar luego en sus URLS y extraer la información'''

for page in pages:
    if page == 1:
        print("Pagina",page)
        URL = url
        response = requests.get(url)
        soup = bs(response.text, 'lxml')
        productos = soup.find_all(class_='caption')
        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)
        
    else:
        print("Pagina",page)
        URL = url+'page' + str(page)+'/'
        response = requests.get(URL)
        soup = bs(response.text, 'lxml')
        productos = soup.find_all(class_='caption')
        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)


for i in range(len(lista_URLs)):
    id.append(i)

    
'''Extraemos la información de cada producto existente'''

for URL in lista_URLs:
    url_product=URL
    response_product = requests.get(url_product)
    soup_product = bs(response_product.text, 'lxml')
    user_comments_product=[]
    date_comments_product=[]
    comments_product=[]
    rating=[]

    titulos=soup_product.find_all("h1",class_="h3")
    for titulo in titulos:
        nombre=titulo.get_text(strip=True)
        name.append(nombre)

    all_price = soup_product.find_all("div", class_="productoPrecio pull-right tdd_precio")                        
    for price_container in all_price:                                                                    
        try:
            special_price = price_container.find("span", class_="productSpecialPrice")
            if special_price:
                item_price = float(special_price.get_text(strip=True).replace(",", ".").split('€')[0])
                new_price.append(item_price)
                regular_price = price_container.find("del").get_text(strip=True)
                item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                regular_prices.append(item_regular_price)
            else:
                regular_price = price_container.find("span").get_text(strip=True)
                item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                new_price.append(item_regular_price)
                regular_prices.append(None)
        except:
            new_price.append(None)
            regular_prices.append(None)

    description=soup_product.find("div", class_="description") 
    information=description.get_text().split('\n')[1:]
    documentation = ''.join(information)
    info.append(documentation)


    '''Vamos a obtener los datos de los comentarios de los usuarios'''

    all_user_comments = soup_product.find_all("span", class_="name-user") 
    for user_comment in all_user_comments:
        user_comments_product.append(user_comment.get_text(strip=True))
    user_comments.append(user_comments_product)

    all_dates = soup_product.find_all("span", class_="date")  
    for dates in all_dates:
        dates_text=dates.get_text(strip=True)
        # dates=datetime.strftime(dates, '%dd/%mm/%Y')
        date_comments_product.append(dates_text)
        # date_object = datetime.strptime(date_comments_product)
    date.append(date_comments_product)

    all_comments = soup_product.find_all("p")
    for formats in all_comments[-len(date_comments_product):]:
        comments_product.append(formats.get_text(strip=True))
    comment.append(comments_product)

    hearts = soup_product.find_all('div', class_= 'box-description')
    for heart in hearts:
        heart_rating = heart.find_all('span', class_= 'fas fa-heart')
        num_hearts = len(heart_rating)
        rating.append(num_hearts)
    ratings.append(rating)

    datos = list(zip( date_comments_product,rating, user_comments_product,comments_product ))
    comentarios.append(datos)

for i, regular_price in enumerate(regular_prices):
    if regular_price is None:
        regular_prices[i] = new_price[i]

'''Generamos un dataframe con este diccionario para poder trabajar con los mismos y generar una nueva estructura para poder trabajar con ella'''

comentarios_productos=pd.DataFrame(diccionario_comentarios_productos)

id=[]
comments=[]
date=[]
ratio=[]
users=[]
comment=[]

comentarios=pd.DataFrame()
diccionario={"id":id,"comments":comments}

for id_product,n_comments in enumerate (comentarios_productos['COMENTARIOS']):
    # print("Imprimiendo texto del indice",id_product)
    # print("Imprimiento n_comentarios",len(n_comments))
    for i in n_comments:
        # print("id",id_product,"coments",comments)
        id.append(id_product)
        comments.append(i)


for j in range(len(diccionario['comments'])):
    date.append(diccionario['comments'][j][0])
    ratio.append(diccionario['comments'][j][1])
    users.append(diccionario['comments'][j][2])
    comment.append(diccionario['comments'][j][3])


comentarios['ID']=pd.Series(id)
comentarios['DATE']=pd.Series(date)
comentarios['RATIO']=pd.Series(ratio)
comentarios['USERS']=pd.Series(users)
comentarios['COMMENT']=pd.Series(comment)
comentarios.info()

Pagina 1
Pagina 2
Pagina 3
Pagina 4
Pagina 5
Pagina 6
Pagina 7
Pagina 8
Pagina 9
Pagina 10
Pagina 11
Pagina 12
Pagina 13
Pagina 14
Pagina 15
Pagina 16
Pagina 17
Pagina 18
Pagina 19
Pagina 20
Pagina 21
Pagina 22
Pagina 23
Pagina 24
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11551 entries, 0 to 11550
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID       11551 non-null  int64 
 1   DATE     11551 non-null  object
 2   RATIO    11551 non-null  int64 
 3   USERS    11551 non-null  object
 4   COMMENT  11551 non-null  object
dtypes: int64(2), object(3)
memory usage: 451.3+ KB


Vamos a visualizar los datos que existen en los dos diccionarios.

Para ello pasaremos el diccionario de productos a un Dataframe y después haremos un estudio de los mismos.

Vamos a ver si cuántos duplicados hay en este Dataframe, ya que navegando por la web hemos visto algún producto repetido.

In [3]:
productos=pd.DataFrame(diccionario_datos_productos)
productos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              576 non-null    int64  
 1   NAME            576 non-null    object 
 2   INFO            576 non-null    object 
 3   LISTA_URL       576 non-null    object 
 4   REGULAR_PRICE   576 non-null    float64
 5   DISCOUNT_PRICE  576 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 27.1+ KB


In [4]:
comentarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11551 entries, 0 to 11550
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID       11551 non-null  int64 
 1   DATE     11551 non-null  object
 2   RATIO    11551 non-null  int64 
 3   USERS    11551 non-null  object
 4   COMMENT  11551 non-null  object
dtypes: int64(2), object(3)
memory usage: 451.3+ KB


In [5]:
productos.NAME.value_counts()

OISHi, Camiseta de rejilla negra con mangas                     2
WARRIOR, suave casco vibrador para masaje integral              2
Columpio sexual de amantis, el asistente postural definitivo    2
GALGA, sofisticadas braguitas-arnés de amantis                  2
DOBERMANA, arnés fino y de doble anilla                         2
                                                               ..
SAREA, body de calentamiento de rejilla ancha                   1
JUSTISSE CHOCKER, unisex con argolla                            1
ADA, Vagina translúcida extra suave de amantis                  1
LOCOMOTION rugged, tu sex-machine con vibración y ventosa       1
JUSTISSE BACK - mordaza de bola con esposas                     1
Name: NAME, Length: 550, dtype: int64

#### Eliminando duplicados de los dos dataframes.

In [6]:
noduplicated_product = productos.drop_duplicates(subset='NAME', keep='first')
noduplicated_product.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 550 entries, 0 to 575
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              550 non-null    int64  
 1   NAME            550 non-null    object 
 2   INFO            550 non-null    object 
 3   LISTA_URL       550 non-null    object 
 4   REGULAR_PRICE   550 non-null    float64
 5   DISCOUNT_PRICE  550 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 30.1+ KB


In [7]:
removed_id = productos[productos.duplicated(subset='NAME', keep='first')]['ID']
noduplicated_comments = comentarios[~comentarios['ID'].isin(productos[productos['ID'].isin(removed_id)]['ID'])]
noduplicated_comments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10659 entries, 0 to 11550
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID       10659 non-null  int64 
 1   DATE     10659 non-null  object
 2   RATIO    10659 non-null  int64 
 3   USERS    10659 non-null  object
 4   COMMENT  10659 non-null  object
dtypes: int64(2), object(3)
memory usage: 499.6+ KB


#### Generamos nuevas variables dentro del dataframe de **productos**.

Vamos a crear 2 nuevas columnas a partir de *Name* y *Description*, donde dejaremos el nombre del producto y su slogan por un lado y por otro la descripción y las características por otro.

En principio son variables que no necesitaremos para estos estudios pero las guardaremos por si hay que hacer algún estudio posterior de las mismas (comparativas de productos similares de esta página web o de otras páginas web, por ejemplo).

In [8]:
noduplicated_product['NAME'] = noduplicated_product['NAME'].str.replace(r'-(?=\w)', '_')
noduplicated_product[['PRODUCT', 'SLOGAN']] = noduplicated_product['NAME'].str.split('[,-.]', 1, expand=True)
noduplicated_product['PRODUCT'] = noduplicated_product['PRODUCT'].str.strip()
noduplicated_product['SLOGAN'] = noduplicated_product['SLOGAN'].str.strip()
noduplicated_product.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 550 entries, 0 to 575
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              550 non-null    int64  
 1   NAME            550 non-null    object 
 2   INFO            550 non-null    object 
 3   LISTA_URL       550 non-null    object 
 4   REGULAR_PRICE   550 non-null    float64
 5   DISCOUNT_PRICE  550 non-null    float64
 6   PRODUCT         550 non-null    object 
 7   SLOGAN          454 non-null    object 
dtypes: float64(2), int64(1), object(5)
memory usage: 38.7+ KB


C:\Users\Javier\AppData\Local\Temp\ipykernel_16032\3431089402.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  noduplicated_product['NAME'] = noduplicated_product['NAME'].str.replace(r'-(?=\w)', '_')
C:\Users\Javier\AppData\Local\Temp\ipykernel_16032\3431089402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noduplicated_product['NAME'] = noduplicated_product['NAME'].str.replace(r'-(?=\w)', '_')
C:\Users\Javier\AppData\Local\Temp\ipykernel_16032\3431089402.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  noduplicated_product[['PRODUCT', 'SLOGAN']] = noduplicated_product['NAME'].str.split('[,-.]', 1,

Vamos a realizar un visionado de los datos que tienen para discernir donde se puede realizar la separación.

Por un lado extraeremos las características en una nueva columna, dejando la Description en la misma. 

In [9]:
noduplicated_product['CHARACTERISTICS'] = noduplicated_product['INFO'].str.split('Ver características y medidas|Características', 1).str[1]
noduplicated_product['DESCRIPTION'] = noduplicated_product['INFO'].str.split('Ver características y medidas|Características', 1).str[0].str.strip()
noduplicated_product['CHARACTERISTICS'] = noduplicated_product['CHARACTERISTICS'].str.replace('\r', ' ')
noduplicated_product['DESCRIPTION'] = noduplicated_product['DESCRIPTION'].str.replace('\r', ' ')
noduplicated_product.info()

C:\Users\Javier\AppData\Local\Temp\ipykernel_16032\1090549632.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  noduplicated_product['CHARACTERISTICS'] = noduplicated_product['INFO'].str.split('Ver características y medidas|Características', 1).str[1]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 550 entries, 0 to 575
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               550 non-null    int64  
 1   NAME             550 non-null    object 
 2   INFO             550 non-null    object 
 3   LISTA_URL        550 non-null    object 
 4   REGULAR_PRICE    550 non-null    float64
 5   DISCOUNT_PRICE   550 non-null    float64
 6   PRODUCT          550 non-null    object 
 7   SLOGAN           454 non-null    object 
 8   CHARACTERISTICS  489 non-null    object 
 9   DESCRIPTION      550 non-null    object 
dtypes: float64(2), int64(1), object(7)
memory usage: 47.3+ KB


C:\Users\Javier\AppData\Local\Temp\ipykernel_16032\1090549632.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noduplicated_product['CHARACTERISTICS'] = noduplicated_product['INFO'].str.split('Ver características y medidas|Características', 1).str[1]
C:\Users\Javier\AppData\Local\Temp\ipykernel_16032\1090549632.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  noduplicated_product['DESCRIPTION'] = noduplicated_product['INFO'].str.split('Ver características y medidas|Características', 1).str[0].str.strip()
C:\Users\Javier\AppData\Local\Temp\ipykernel_16032\1090549632.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

Reestructuramos el dataframe

In [10]:
col_1 = noduplicated_product.pop('PRODUCT')
col_2=noduplicated_product.pop('SLOGAN')
col_3=noduplicated_product.pop('DESCRIPTION')
col_4=noduplicated_product.pop('CHARACTERISTICS')

noduplicated_product.drop(columns=['NAME'],inplace=True)
noduplicated_product.drop(columns=['INFO'],inplace=True)

noduplicated_product.insert(loc= 1 , column= 'PRODUCT', value= col_1)
noduplicated_product.insert(loc= 2 , column= 'SLOGAN', value= col_2)
noduplicated_product.insert(loc= 3 , column= 'DESCRIPTION', value= col_3)
noduplicated_product.insert(loc= 4 , column= 'CHARACTERISTICS', value= col_4)
noduplicated_product.head()

C:\Users\Javier\AppData\Local\Temp\ipykernel_16032\2928903985.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noduplicated_product.drop(columns=['NAME'],inplace=True)
C:\Users\Javier\AppData\Local\Temp\ipykernel_16032\2928903985.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noduplicated_product.drop(columns=['INFO'],inplace=True)


,ID,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL,REGULAR_PRICE,DISCOUNT_PRICE
0,0,Desliz! Lubricante íntimo de agua 100ml,None,"Algunos lubricantes son un poco densos, otros ...",Bote de 100ml de venta exclusiva en amantis.ne...,https://www.amantis.net/desliz-lubricante-inti...,9.99,7.99
1,1,TOBOGANE HOT RABBIT,el superventas de amantis ¡mejorado!,Vuelve nuestro vibrador de doble estimulación ...,"Medidas: 19cm (11cm insertables) y 3,3cm/ 2,2c...",https://www.amantis.net/tobogane-hot-rabbit-el...,89.99,39.99
2,2,BALLENATO,tu vibrador a distancia con aleta móvil y sume...,De las profundidades más húmedas llega BALLENA...,Mando a distancia.Peso 62 gramos.Silicona médi...,https://www.amantis.net/ballenato-tu-vibrador-...,99.99,49.99
3,3,TANDEM 2 flex,vibrador doble flexible con mando,Tanto si ya eras fan de nuestro querido TANDEM...,10 patrones de vibración diferentes y tres niv...,https://www.amantis.net/tandem-2-flex-vibrador...,109.99,49.99
4,4,FOXY,Succionador con mango vibrador y orejitas,"Los succionadores van evolucionando, vamos des...",Vibrador con orejitas y succión simultáneaMate...,https://www.amantis.net/foxy-vibrador-succiona...,59.99,39.99


#### Convertimos la feauture *DATE* de *COMENTARIOS*, que es de tipo string en *Datetime*.

De la columna *DATE* extraigo los valores de *DAY*, *MONTH* y *YEAR*.

Con la columna *MONTH* hago un mapeo para convertir en el número correspondiente del mes y así poder generar la nueva columna *DATE* con estos  datos.

Convierto la columna *DATE* en datetime con los datos de las 3 últimas columnas.

In [11]:
'''Este es el diccionario para mapear los meses'''

dm_mapping={
    'enero':1, 
    'febrero':2, 
    'marzo':3, 
    'abril':4, 
    'mayo':5,
    'junio':6, 
    'julio':7,
    'agosto':8, 
    'septiembre':9, 
    'octubre':10, 
    'noviembre':11, 
    'diciembre':12,
} 

In [12]:
noduplicated_comments['DAY']=noduplicated_comments['DATE'].str.split(' ').str.get(1).astype('Int64')
noduplicated_comments['MONTH']=noduplicated_comments['DATE'].str.split(' ').str.get(2).str.split(',').str.get(0)
noduplicated_comments['YEAR']=noduplicated_comments['DATE'].str.split(' ').str.get(-1).astype('Int64')

noduplicated_comments['MONTH']=noduplicated_comments['MONTH'].map(dm_mapping)
noduplicated_comments['DATE'] = pd.to_datetime(noduplicated_comments.iloc[:,-3:])
noduplicated_comments.head()


C:\Users\Javier\AppData\Local\Temp\ipykernel_16032\930944462.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noduplicated_comments['DAY']=noduplicated_comments['DATE'].str.split(' ').str.get(1).astype('Int64')
C:\Users\Javier\AppData\Local\Temp\ipykernel_16032\930944462.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noduplicated_comments['MONTH']=noduplicated_comments['DATE'].str.split(' ').str.get(2).str.split(',').str.get(0)
C:\Users\Javier\AppData\Local\Temp\ipykernel_16032\930944462.py:3: Setti

,ID,DATE,RATIO,USERS,COMMENT,DAY,MONTH,YEAR
0,0,2023-05-18,5,David,"Muy recomendable. El bote cunde, hidrata muy b...",18,5,2023
1,0,2023-04-12,5,Ferran,Es la primera vez que compro este lubricante y...,12,4,2023
2,0,2023-04-10,5,Sergio,He probado varios tipos de lubricante y este d...,10,4,2023
3,0,2023-01-23,5,Jaime,Perfecto para embadurnarte el pincelin y meter...,23,1,2023
4,0,2023-01-11,5,YASMINA,"Buen producto, tanto para el cuerpo a cuerpo c...",11,1,2023


#### Generamos ficheros .csv para ir guardando esta información antes de dejarlas salvadas en una BBDD Relacional.

In [13]:
noduplicated_product.to_csv('./Data/productos.csv',header=True,index=False)                       # Tengo que generar el path correcto
noduplicated_comments.to_csv('./Data/comentarios_scrape.csv',header=True,index=True)           # Tengo que generar el path correcto

## 2. Tratamiento de los datos de texto con RegEx y Librerías de NLP

Primeramente vamos a generar nuevas columnas a partir de *Name* y *Description*. 

Estas columnas las generaremos a partir de *RegEx* y los métodos *str.split* y *str.replace*.

Utilizaremos las siguientes librerías para el tratamiento  de la información.